# Reliability anaysis


## Packages and Helpers

In [40]:
# packages
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from utils import load_dataframes
from scipy.stats import pearsonr
import random
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
import pingouin as pg
import seaborn as sns

## Read Processed Data

In [41]:
# load overall df
random_all_data = pd.read_csv("processed_data/items_per_LLM_random_simulation.csv")
random_all_data_without_turn = pd.read_csv("processed_data/sim_rand_without_turn.csv") #items_per_LLM_random_simulation_without_turn

semi_random_all_data = pd.read_csv("processed_data/items_per_LLM_semi_random_simulation.csv")
semi_random_all_data_without_turn = pd.read_csv("processed_data/items_per_LLM_semi_random_simulation_without_turn.csv")

non_rand_answer_options_data = pd.read_csv("processed_data/sim_non_rand_answer_tendencies.csv")
non_rand_answer_options_data_without_turn = pd.read_csv("processed_data/sim_non_rand_answer_tendencies_without_turn.csv")

## Normalize item scores where tasks have different scales on different items
- AUDIT, FTND, GABS, PG

In [42]:
def normalize_per_scale(data):
    scales_to_normalize = ["AUDIT scale", "FTND scale", "GABS scale", "PG scale"]

    data = data.copy()

    # Loop over each experiment you want to normalize
    for scale in scales_to_normalize:
        mask = data["experiment"] == scale

        if mask.sum() == 0:
            continue  # skip if scale not present

        # Normalize score
        data.loc[mask, "score"] = (
            data.loc[mask, "score"] - data.loc[mask, "score"].min()
        ) / (data.loc[mask, "score"].max() - data.loc[mask, "score"].min())

        # Normalize score_top_n
        data.loc[mask, "score_top_n"] = (
            data.loc[mask, "score_top_n"] - data.loc[mask, "score_top_n"].min()
        ) / (data.loc[mask, "score_top_n"].max() - data.loc[mask, "score_top_n"].min())

    return data

random_all_data = normalize_per_scale(random_all_data)
random_all_data_without_turn = normalize_per_scale(random_all_data_without_turn)
semi_random_all_data = normalize_per_scale(semi_random_all_data)
semi_random_all_data_without_turn = normalize_per_scale(semi_random_all_data_without_turn)
non_rand_answer_options_data = normalize_per_scale(non_rand_answer_options_data)
non_rand_answer_options_data_without_turn = normalize_per_scale(non_rand_answer_options_data_without_turn)


## Chronbach's Alpha

In [43]:
# function:
def compute_cronbach_alpha(data, score = "score"):
    results = []

    # loop through experiments
    for exp, exp_data in data.groupby("experiment"):
        
        # check whether this experiment has subcategories
        if exp_data["category"].notna().any() and exp != "SOEP scale":
            # compute alpha per category (domain)
            for domain, domain_data in exp_data.groupby("category"):
                df_wide = domain_data.pivot_table(
                    index="model", columns="item", values=score
                )
                #print(df_wide)
                if df_wide.shape[1] > 1:
                    alpha, ci = pg.cronbach_alpha(df_wide)
                else:
                    alpha, ci = None, (None, None)
                
                results.append({
                    "experiment": exp,
                    "domain": domain,
                    "alpha": alpha,
                    "alpha_CI": ci
                })
        else:
            # compute alpha for the whole experiment
            df_wide = exp_data.pivot_table(
                index="model", columns="item", values=score
            )
            #print(exp, df_wide.std().describe())
            if df_wide.shape[1] > 1:
                alpha, ci = pg.cronbach_alpha(df_wide)
            else:
                alpha, ci = None, (None, None)
            
            results.append({
                "experiment": exp,
                "domain": "total",  # no subdomain
                "alpha": alpha,
                "alpha_CI": ci
            })

    # convert to DataFrame
    return(pd.DataFrame(results))



In [44]:
# ---- Cronbach's alpha per domain or directly per scale, depending on whether the scale has subdomains ----
alpha_df_random = compute_cronbach_alpha(random_all_data)
alpha_df_random_without_turn = compute_cronbach_alpha(random_all_data_without_turn)
alpha_df_semi_random = compute_cronbach_alpha(semi_random_all_data)
alpha_df_semi_random_without_turn = compute_cronbach_alpha(semi_random_all_data_without_turn)
alpha_df_non_rand_answer_options = compute_cronbach_alpha(non_rand_answer_options_data)
alpha_df_non_rand_answer_options_without_turn = compute_cronbach_alpha(non_rand_answer_options_data_without_turn)


# ---- Cronbach's alpha per domain or directly per scale, depending on whether the scale has subdomains ----
alpha_df_random_top_n= compute_cronbach_alpha(random_all_data, score = "score_top_n")
alpha_df_semi_random_top_n = compute_cronbach_alpha(semi_random_all_data, score = "score_top_n")
alpha_df_non_rand_answer_options_top_n = compute_cronbach_alpha(non_rand_answer_options_data, score = "score_top_n")



# Merge only selected columns
alpha_df = alpha_df_random.merge(
    alpha_df_semi_random,
    on=["experiment", "domain"],
    how="inner",
    suffixes=("_random", "_semi_random")
)

alpha_df_without_turn = alpha_df_random_without_turn.merge(
    alpha_df_semi_random_without_turn,
    on=["experiment", "domain"],
    how="inner",
    suffixes=("_random", "_semi_random")
)

alpha_df = alpha_df.merge(
    alpha_df_without_turn,
    on=["experiment", "domain"],
    how="inner",
    suffixes=("", "_without_flipping")
)

alpha_df_non_r = alpha_df_non_rand_answer_options.merge(
    alpha_df_non_rand_answer_options_without_turn,
    on=["experiment", "domain"], 
    how="inner",
    suffixes=("_flipped", "_without_flipping")
)

display(alpha_df)
display(alpha_df_non_r)

,experiment,domain,alpha_random,alpha_CI_random,alpha_semi_random,alpha_CI_semi_random,alpha_random_without_flipping,alpha_CI_random_without_flipping,alpha_semi_random_without_flipping,alpha_CI_semi_random_without_flipping
0,AUDIT scale,total,0.185702,"[-0.213, 0.495]",0.125090,"[-0.303, 0.457]",-0.280847,"[-0.908, 0.205]",0.953454,"[0.931, 0.971]"
1,BARRAT scale,BISa,0.085305,"[-0.377, 0.436]",-0.043888,"[-0.571, 0.357]",0.166357,"[-0.255, 0.486]",0.990053,"[0.985, 0.994]"
2,BARRAT scale,BISm,0.188272,"[-0.209, 0.496]",-0.241641,"[-0.849, 0.23]",-0.206726,"[-0.797, 0.251]",0.991035,"[0.987, 0.994]"
3,BARRAT scale,BISn,0.014077,"[-0.469, 0.388]",0.200345,"[-0.191, 0.504]",-0.314574,"[-0.958, 0.184]",0.998668,"[0.998, 0.999]"
4,BART task,total,0.127238,"[-0.29, 0.459]",0.127238,"[-0.29, 0.459]",0.161147,"[-0.24, 0.48]",0.982769,"[0.975, 0.989]"
5,CARE scale,CAREa,0.083517,"[-0.373, 0.434]",-0.016770,"[-0.524, 0.372]",-0.016770,"[-0.524, 0.372]",0.891735,"[0.838, 0.933]"
6,CARE scale,CAREs,0.095543,"[-0.38, 0.447]",-0.040084,"[-0.586, 0.364]",-0.040084,"[-0.586, 0.364]",0.599966,"[0.39, 0.755]"
7,CARE scale,CAREw,0.230793,"[-0.21, 0.539]",-0.237210,"[-0.945, 0.258]",-0.237210,"[-0.945, 0.258]",0.779589,"[0.653, 0.868]"
8,CCT task,total,-0.124240,"[-0.697, 0.33]",-0.124240,"[-0.697, 0.33]",-0.123541,"[-0.696, 0.33]",0.984221,"[0.976, 0.991]"
9,DAST scale,total,-0.198741,"[-0.765, 0.251]",-0.286199,"[-0.894, 0.196]",0.174552,"[-0.216, 0.484]",0.999941,"[1.0, 1.0]"


,experiment,domain,alpha_flipped,alpha_CI_flipped,alpha_without_flipping,alpha_CI_without_flipping
0,AUDIT scale,total,0.884494,"[0.828, 0.928]",0.953454,"[0.931, 0.971]"
1,BARRAT scale,BISa,0.971975,"[0.958, 0.983]",0.990053,"[0.985, 0.994]"
2,BARRAT scale,BISm,0.937814,"[0.907, 0.961]",0.991035,"[0.987, 0.994]"
3,BARRAT scale,BISn,0.885266,"[0.829, 0.929]",0.998668,"[0.998, 0.999]"
4,BART task,total,0.982769,"[0.975, 0.989]",0.982769,"[0.975, 0.989]"
5,CARE scale,CAREa,0.891735,"[0.838, 0.933]",0.891735,"[0.838, 0.933]"
6,CARE scale,CAREs,0.599966,"[0.39, 0.755]",0.599966,"[0.39, 0.755]"
7,CARE scale,CAREw,0.779589,"[0.653, 0.868]",0.779589,"[0.653, 0.868]"
8,CCT task,total,0.984221,"[0.976, 0.991]",0.984221,"[0.976, 0.991]"
9,DAST scale,total,0.043050,"[-0.409, 0.402]",0.999941,"[1.0, 1.0]"


## check correlations of semi-random data

In [ ]:
#df_wide.describe()

item,3,11,16,17,20,21,23,28,38,40
count,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000
mean,1.546034,1.676008,1.718796,1.843790,1.612622,1.682557,1.711236,1.524975,1.724209,1.577981
std,0.012732,0.008296,0.010031,0.009202,0.011328,0.008974,0.011573,0.011646,0.007743,0.011206
min,1.523367,1.658535,1.698003,1.826437,1.581424,1.667001,1.689403,1.504889,1.709267,1.557175
25%,1.534944,1.671121,1.712130,1.837083,1.604017,1.675559,1.704304,1.518411,1.718826,1.570257
50%,1.546021,1.674683,1.717884,1.843928,1.614492,1.684744,1.708832,1.524497,1.725840,1.579787
75%,1.554330,1.681922,1.726799,1.850877,1.620993,1.690303,1.721218,1.532348,1.730325,1.587373
max,1.574378,1.694683,1.739475,1.863585,1.635401,1.701530,1.734670,1.553443,1.737883,1.595572


In [46]:
df_wide = non_rand_answer_options_data[non_rand_answer_options_data["category"] == "SStas"].pivot_table(index='model', columns='item', values='score')
sorted_cols = sorted(df_wide.columns, key=lambda x: int(x))
df_wide = df_wide[sorted_cols]
correlations = df_wide.corr()
correlations

item,3,11,16,17,20,21,23,28,38,40
item,,,,,,,,,,
3,1.000000,-0.697319,0.820694,0.743744,-0.684607,-0.724496,0.751678,0.823760,-0.663179,-0.769016
11,-0.697319,1.000000,-0.658476,-0.785659,0.725821,0.693214,-0.638508,-0.768482,0.684901,0.734145
16,0.820694,-0.658476,1.000000,0.789442,-0.726900,-0.696952,0.818881,0.828781,-0.678776,-0.751506
17,0.743744,-0.785659,0.789442,1.000000,-0.658834,-0.754506,0.807979,0.779418,-0.618769,-0.780702
20,-0.684607,0.725821,-0.726900,-0.658834,1.000000,0.710066,-0.670887,-0.719684,0.630122,0.712596
21,-0.724496,0.693214,-0.696952,-0.754506,0.710066,1.000000,-0.830385,-0.735699,0.680906,0.749584
23,0.751678,-0.638508,0.818881,0.807979,-0.670887,-0.830385,1.000000,0.759244,-0.694234,-0.783794
28,0.823760,-0.768482,0.828781,0.779418,-0.719684,-0.735699,0.759244,1.000000,-0.714527,-0.739118
38,-0.663179,0.684901,-0.678776,-0.618769,0.630122,0.680906,-0.694234,-0.714527,1.000000,0.629332


## Split-half Reliability

In [47]:
def split_half_reliability(df_items, n_splits=100):
    """Compute average split-half reliability (Spearman-Brown corrected)."""

    k = df_items.shape[1]
    
    if k < 2:
        return None, None

    cols = df_items.columns
    results = []

    for _ in range(n_splits):
        shuffled = np.random.permutation(cols)
        half1 = shuffled[:k//2]
        half2 = shuffled[k//2:]

        s1 = df_items[half1].sum(axis=1)
        s2 = df_items[half2].sum(axis=1)

        r, _ = pearsonr(s1, s2)

        if np.isfinite(r):
            r_sb = (2 * r) / (1 + r)  # Spearman-Brown correction
            results.append(r_sb)

    if len(results) == 0:
        return None, None
    
    return np.mean(results), np.std(results)


In [48]:
def compute_split_half_reliability(data, score="score"):
    rows = []  # final list of results

    for exp, exp_data in data.groupby("experiment"):

        # case 1: experiments with domains
        if exp_data["category"].notna().any() and exp != "SOEP scale":
            for domain, domain_data in exp_data.groupby("category"):

                df_wide = domain_data.pivot_table(
                    index="model", columns="item", values=score
                )

                mean_rsb, sd_rsb = split_half_reliability(df_wide)

                rows.append({
                    "experiment": exp,
                    "domain": domain,
                    "split_half_mean": mean_rsb,
                    "split_half_sd": sd_rsb
                })

        # case 2: experiments without domains
        else:
            df_wide = exp_data.pivot_table(
                index="model", columns="item", values=score
            )

            mean_rsb, sd_rsb = split_half_reliability(df_wide)

            rows.append({
                "experiment": exp,
                "domain": "total",
                "split_half_mean": mean_rsb,
                "split_half_sd": sd_rsb
            })

    return pd.DataFrame(rows)


In [49]:
split_half_reliability_random = compute_split_half_reliability(random_all_data)
split_half_reliability_random_without_turn = compute_split_half_reliability(random_all_data_without_turn)
split_half_reliability_semi_random = compute_split_half_reliability(semi_random_all_data)
split_half_reliability_semi_random_without_turn = compute_split_half_reliability(semi_random_all_data_without_turn)
split_half_non_rand_answer_options = compute_split_half_reliability(non_rand_answer_options_data)
split_half_non_rand_answer_options_without_turn = compute_split_half_reliability(non_rand_answer_options_data_without_turn)


# Merge 
split_half_rel_df = split_half_reliability_random.merge(
    split_half_reliability_semi_random,
    on=["experiment", "domain"],
    how="inner",
    suffixes=("_random", "_semi_random")
)
split_half_rel_df_without_flipping = split_half_reliability_random_without_turn.merge(
    split_half_reliability_semi_random_without_turn,
    on=["experiment", "domain"],
    how="inner",
    suffixes=("_random", "_semi_random")
)
split_half_rel_df = split_half_rel_df.merge(
    split_half_rel_df_without_flipping,
    on=["experiment", "domain"],
    how="inner",
    suffixes=("", "_without_flipping")
)

split_half_rel_df_non_rand = split_half_non_rand_answer_options.merge(
    split_half_non_rand_answer_options_without_turn,
    on=["experiment", "domain"],
    how="inner",
    suffixes=("_flipped", "_without_flipping")
)

display(split_half_rel_df)
display(split_half_rel_df_non_rand)

,experiment,domain,split_half_mean_random,split_half_sd_random,split_half_mean_semi_random,split_half_sd_semi_random,split_half_mean_random_without_flipping,split_half_sd_random_without_flipping,split_half_mean_semi_random_without_flipping,split_half_sd_semi_random_without_flipping
0,AUDIT scale,total,0.196485,0.174013,0.130547,0.188038,-0.277735,0.244964,0.977750,0.016365
1,BARRAT scale,BISa,0.091446,0.146951,-0.042473,0.203725,0.148315,0.174224,0.996366,0.004331
2,BARRAT scale,BISm,0.205723,0.166450,-0.229589,0.235316,-0.204635,0.251565,0.996525,0.004162
3,BARRAT scale,BISn,0.034486,0.173266,0.188210,0.145970,-0.298401,0.258392,0.999361,0.000517
4,BART task,total,0.120414,0.226044,0.121032,0.226453,0.182732,0.163388,0.997312,0.000753
5,CARE scale,CAREa,0.111212,0.134579,0.013348,0.148294,-0.037992,0.132040,0.956151,0.025656
6,CARE scale,CAREs,-0.025536,0.252711,-0.161777,0.280452,-0.100430,0.194445,0.863051,0.055844
7,CARE scale,CAREw,0.176996,0.349842,-0.277114,0.147256,-0.277233,0.144243,0.886308,0.055367
8,CCT task,total,-0.094727,0.263559,-0.118361,0.267807,-0.117058,0.241922,0.999298,0.000717
9,DAST scale,total,-0.188271,0.191562,-0.292172,0.241546,0.192901,0.166834,0.999944,0.000019


,experiment,domain,split_half_mean_flipped,split_half_sd_flipped,split_half_mean_without_flipping,split_half_sd_without_flipping
0,AUDIT scale,total,0.953561,0.044382,0.975610,0.021506
1,BARRAT scale,BISa,0.993584,0.003215,0.996342,0.003958
2,BARRAT scale,BISm,0.992474,0.004557,0.996357,0.004352
3,BARRAT scale,BISn,0.988197,0.008460,0.999343,0.000562
4,BART task,total,0.997241,0.000708,0.997255,0.000615
5,CARE scale,CAREa,0.945301,0.030177,0.947804,0.030688
6,CARE scale,CAREs,0.874082,0.049839,0.863992,0.057053
7,CARE scale,CAREw,0.893197,0.053044,0.889755,0.054329
8,CCT task,total,0.999248,0.000869,0.999307,0.000752
9,DAST scale,total,-0.199065,1.092392,0.999940,0.000025
